## Work bench to improve parsing and updating laws
This notebook is targeted to "d18-3823.pdf" and the change to the Berliner Straßengesetz.

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lawinprogress.apply_changes.apply_changes import apply_changes
from lawinprogress.parsing.change_law_utils import preprocess_raw_law
from lawinprogress.parsing.parse_change_law import (
    parse_change_request_line,
    parse_change_law_tree,
)
from lawinprogress.parsing.parse_source_law import parse_source_law_tree
from lawinprogress.parsing.lawtree import LawTextNode
from lawinprogress.parsing.proposal_pdf_to_artikles import (
    extract_law_titles,
    extract_raw_proposal,
    extract_seperate_change_proposals,
    read_pdf_law,
    remove_inkrafttreten,
)

### Read and process the change law

In [3]:
change_law_path = "../data/d18-3823.pdf"

In [4]:
change_law_raw_pages = read_pdf_law(change_law_path)

# idenfify the different laws affected
change_law_extract = extract_raw_proposal(change_law_raw_pages)
proposals_list = extract_seperate_change_proposals(change_law_extract)
law_titles = extract_law_titles(proposals_list)
law_titles, proposals_list = remove_inkrafttreten(law_titles, proposals_list)

In [5]:
law_titles

['Berliner Straßengesetzes',
 'Allgemeinen Zuständigkeitsgesetzes    Nummer',
 'Allgemeinen Sicherheits- und Ordnungsgesetzes',
 'Verordnung über die Zuständigkeit für einzelne Bezirksaufgaben',
 'Berliner Mobilitätsgesetzes']

In [6]:
# we only work with the first one here
change_law = proposals_list[0]
# << do some dirty fix here, since someone forgot to add '' to the change law...
change_law = "Stelle abgewickelt werden.“".join(
    change_law.split("Stelle abgewickelt werden.")
)
print(change_law)

Änderung des Berliner Straßengesetzes 
 
Das Berliner Straßengesetz vom 13. Juli 1999 (GVBl. S. 380), das zuletzt durch Artikel 27 des 
Gesetzes vom 12. Oktober 2020 (GVBl. S. 807) geändert worden ist, wird wie folgt geändert: 
 
1. In der Inhaltsübersicht wird nach der Angabe zu § 11 folgende Angabe eingefügt: 
 
„§ 11a Sondernutzung für das gewerbliche Anbieten von Mietfahrzeugen“. 
 
2. In § 2 Absatz 2 Nummer 1 Buchstabe b wird das Wort „Grünanlagen“ durch das Wort 
„Straßenbegleitgrün“ ersetzt. 
 
3. In § 5 Absatz 1 Satz 2 werden die Wörter „auf Antrag und“ gestrichen. 
 
4. § 7 wird wie folgt geändert: 
    a)    Nach Absatz 6 wird folgender Absatz 7 eingefügt: 
 
„(7) Soweit der Straßenbaulastträger Maßnahmen, die über das dem regelmäßigen   
Verkehrsbedürfnis entsprechende Maß nach Absatz 2 hinausgehen, auf Veranlassung 
einer anderen Person durchführt, trägt diese Person die Kosten. Das gilt nicht für 
  4 
Haltestellenbereiche  für  den  Linienverkehr.  Der  Träger  der  Straß

In [8]:
# parse source law
clean_change_law = preprocess_raw_law(change_law)
parsed_change_law_tree = LawTextNode(text=law_titles[0], bulletpoint="change")
parsed_change_law_tree = parse_change_law_tree(
    text=clean_change_law, source_node=parsed_change_law_tree
)

In [9]:
# clean_change_law

In [10]:
parsed_change_law_tree._print()

change - Berliner Straßengesetzes
├── 1. - In der Inhaltsübersicht wird nach der Angabe zu § 11 folgende Angabe eingefügt:„§ 11a Sondernutzung für das gewerbliche Anbieten von Mietfahrzeugen“.
├── 2. - In § 2 Absatz 2 Nummer 1 Buchstabe b wird das Wort „Grünanlagen“ durch das Wort„Straßenbegleitgrün“ ersetzt.
├── 3. - In § 5 Absatz 1 Satz 2 werden die Wörter „auf Antrag und“ gestrichen.
├── 4. - § 7 wird wie folgt geändert:
│   ├── a) - Nach Absatz 6 wird folgender Absatz 7 eingefügt:„(7) Soweit der Straßenbaulastträger Maßnahmen, die über das dem regelmäßigen    Verkehrsbedürfnis entsprechende Maß nach Absatz 2 hinausgehen, auf Veranlassung  einer anderen Person durchführt, trägt diese Person die Kosten. Das gilt nicht für    4  Haltestellenbereiche  für  den  Linienverkehr.  Der  Träger  der  Straßenbaulast  kann  angemessene  Vorauszahlungen  oder  Sicherheitsleistungen  verlangen.  §  9  bleibt  unberührt.“
│   └── b) - Der bisherige Absatz 7 wird Absatz 8.
├── 5. - § 9 wird wie fo

In [11]:
# build up a list of texts that describe a single change
all_change_lines = []
for l in parsed_change_law_tree.leaves:
    path = [str(l)]
    node = l
    while node.parent:
        node = node.parent
        path.append(str(node))
    change_line = " ".join(path[::-1][1:])
    all_change_lines.append(change_line)

In [12]:
# parse the change requests in a structured format
change_requests = []
for change_request_line in all_change_lines:
    res = parse_change_request_line(change_request_line)
    if res:
        change_requests.extend(res)

In [13]:
all_change_lines[-2]

'13. - § 28 Absatz 1 wird wie folgt geändert: e) - Die bisherige Nummer 5 wird Nummer 6 und es werden nach dem Wort „Kennzeichen“die Wörter „, gültige Versicherungskennzeichen oder gültige Versicherungsplaketten“eingefügt.'

In [14]:
change_requests

[Change(location=['Inhaltsübersicht', '§ 11'], sentences=[], text=['§ 11a Sondernutzung für das gewerbliche Anbieten von Mietfahrzeugen'], change_type='insert_after'),
 Change(location=['§ 2', 'Absatz 2', 'Nummer 1', 'Buchstabe b'], sentences=[], text=['Grünanlagen', 'Straßenbegleitgrün'], change_type='replace'),
 Change(location=['§ 5', 'Absatz 1'], sentences=['Satz 2'], text=['auf Antrag und'], change_type='delete_after'),
 Change(location=['§ 7', 'Absatz 6'], sentences=[], text=['(7) Soweit der Straßenbaulastträger Maßnahmen, die über das dem regelmäßigen    Verkehrsbedürfnis entsprechende Maß nach Absatz 2 hinausgehen, auf Veranlassung  einer anderen Person durchführt, trägt diese Person die Kosten. Das gilt nicht für    4  Haltestellenbereiche  für  den  Linienverkehr.  Der  Träger  der  Straßenbaulast  kann  angemessene  Vorauszahlungen  oder  Sicherheitsleistungen  verlangen.  §  9  bleibt  unberührt.'], change_type='insert_after'),
 Change(location=['§ 7'], sentences=[], text=[

### Process the source law

In [15]:
law_name = "Berliner Straßengesetz"
source_law_path = "../data/source_laws/{}.txt".format(law_name)

In [16]:
# read the source law
with open(source_law_path, "r") as file:
    source_law_text = file.read()

In [17]:
# parse source law
parsed_law_tree = LawTextNode(text=law_name, bulletpoint="source")
parsed_law_tree = parse_source_law_tree(
    text=source_law_text, source_node=parsed_law_tree
)

In [18]:
parsed_law_tree._print()

source - Berliner Straßengesetz
├── § 1 - BerlStrG – Geltungsbereich

Das Gesetz regelt die Rechtsverhältnisse der öffentlichen Straßen. Für Bundesfernstraßen und für Privatstraßen gilt es nur, soweit dies im Folgenden ausdrücklich bestimmt ist.

› zum Seitenbeginn  › zur Einzelansicht
├── § 2 - BerlStrG – Öffentliche Straßen
│   ├── (1) - Öffentliche Straßen im Sinne dieses Gesetzes sind Straßen, Wege und Plätze, die dem öffentlichen Verkehr gewidmet sind.
│   └── (2) - Zur öffentlichen Straße gehören
│       ├── 1. - der Straßenkörper; das sind insbesondere
│       │   ├── a) - der Untergrund, der Unterbau, der Oberbau, Brücken, Tunnel, Durchlässe, Dämme, Gräben, Böschungen, Stützbauwerke, Treppenanlagen, Lärmschutzanlagen, Straßenentwässerungs- und Straßenbeleuchtungsanlagen,
│       │   └── b) - Fahrbahnen, Gehwege, Radwege, Bushaltebuchten, Taxihalteplätze, Parkflächen einschließlich der Parkhäuser, Grünanlagen sowie Trenn-, Seiten-, Rand- und Sicherheitsstreifen,
│       ├── 2. -

In [19]:
print(parsed_law_tree.to_text())

source Berliner Straßengesetz
    § 1 BerlStrG – Geltungsbereich

Das Gesetz regelt die Rechtsverhältnisse der öffentlichen Straßen. Für Bundesfernstraßen und für Privatstraßen gilt es nur, soweit dies im Folgenden ausdrücklich bestimmt ist.

› zum Seitenbeginn  › zur Einzelansicht
    § 2 BerlStrG – Öffentliche Straßen
        (1) Öffentliche Straßen im Sinne dieses Gesetzes sind Straßen, Wege und Plätze, die dem öffentlichen Verkehr gewidmet sind.
        (2) Zur öffentlichen Straße gehören
            1. der Straßenkörper; das sind insbesondere
                a) der Untergrund, der Unterbau, der Oberbau, Brücken, Tunnel, Durchlässe, Dämme, Gräben, Böschungen, Stützbauwerke, Treppenanlagen, Lärmschutzanlagen, Straßenentwässerungs- und Straßenbeleuchtungsanlagen,
                b) Fahrbahnen, Gehwege, Radwege, Bushaltebuchten, Taxihalteplätze, Parkflächen einschließlich der Parkhäuser, Grünanlagen sowie Trenn-, Seiten-, Rand- und Sicherheitsstreifen,
            2. der Luftraum über

### Update the source law

In [20]:
res_law_tree = apply_changes(parsed_law_tree, change_requests)

Location Inhaltsübersicht not found.
No path found in Change(location=['Inhaltsübersicht', '§ 11'], sentences=[], text=['§ 11a Sondernutzung für das gewerbliche Anbieten von Mietfahrzeugen'], change_type='insert_after'). SKIPPING
APPLIED replace: Change(location=['§ 2', 'Absatz 2', 'Nummer 1', 'Buchstabe b'], sentences=[], text=['Grünanlagen', 'Straßenbegleitgrün'], change_type='replace')
APPLIED delete_after: Change(location=['§ 5', 'Absatz 1'], sentences=['Satz 2'], text=['auf Antrag und'], change_type='delete_after')
APPLIED insert_after: Change(location=['§ 7', 'Absatz 6'], sentences=[], text=['(7) Soweit der Straßenbaulastträger Maßnahmen, die über das dem regelmäßigen    Verkehrsbedürfnis entsprechende Maß nach Absatz 2 hinausgehen, auf Veranlassung  einer anderen Person durchführt, trägt diese Person die Kosten. Das gilt nicht für    4  Haltestellenbereiche  für  den  Linienverkehr.  Der  Träger  der  Straßenbaulast  kann  angemessene  Vorauszahlungen  oder  Sicherheitsleistunge

In [21]:
print(res_law_tree.to_text())

source Berliner Straßengesetz
    § 1 BerlStrG – Geltungsbereich

Das Gesetz regelt die Rechtsverhältnisse der öffentlichen Straßen. Für Bundesfernstraßen und für Privatstraßen gilt es nur, soweit dies im Folgenden ausdrücklich bestimmt ist.

› zum Seitenbeginn  › zur Einzelansicht
    § 2 BerlStrG – Öffentliche Straßen
        (1) Öffentliche Straßen im Sinne dieses Gesetzes sind Straßen, Wege und Plätze, die dem öffentlichen Verkehr gewidmet sind.
        (2) Zur öffentlichen Straße gehören
            1. der Straßenkörper; das sind insbesondere
                a) der Untergrund, der Unterbau, der Oberbau, Brücken, Tunnel, Durchlässe, Dämme, Gräben, Böschungen, Stützbauwerke, Treppenanlagen, Lärmschutzanlagen, Straßenentwässerungs- und Straßenbeleuchtungsanlagen,
                b) Fahrbahnen, Gehwege, Radwege, Bushaltebuchten, Taxihalteplätze, Parkflächen einschließlich der Parkhäuser, Straßenbegleitgrün sowie Trenn-, Seiten-, Rand- und Sicherheitsstreifen,
            2. der Luftra